In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from dataclasses import dataclass
from IPython.display import display, Markdown
from plotly.subplots import make_subplots
from sklearn.metrics import auc
from typing import List, Tuple

In [ ]:
# user constants
COMPARISON_MODE = False                # wherher to show graphs of two models comparison or graphs for a single model
CSVS_ROOT = ["...", "..."]             # directories with csvs generated by ./generate_csvs_for_report.py
MODEL_NAME = ["...", "..."]            # names of the evaluated models (only for naming in graphs)
MIN_ISLET_SIZE = 0                     # [0, 50, 100, 150, 200]

In [ ]:
"""
is = incorrectly separated
fn = false negative
fp = false positive
"""
IMAGE_CSV_FILE_NAME = "min{}-image-stats.csv".format(MIN_ISLET_SIZE)
MATCHED_ISLETS_CSV_FILE_NAME = "min{}-matched-islets-stats.csv".format(MIN_ISLET_SIZE)
IS_ISLETS_CSV_FILE_NAME = "min{}-incorrectly-separated-islets-stats.csv".format(MIN_ISLET_SIZE)
FN_ISLETS_CSV_FILE_NAME = "min{}-false-negative-islets-stats.csv".format(MIN_ISLET_SIZE)
FP_ISLETS_CSV_FILE_NAME = "min{}-false-positive-islets-stats.csv".format(MIN_ISLET_SIZE)

MIN_ISLET_SIZES = [0, 50, 100, 150, 200, 250]
ISLET_SIZES_LABELS = ["0 - 50 μm", "50 - 100 μm", "100 - 150 μm", "150 - 200 μm", "200 - 250 μm", "> 250 μm"]
GT_NN_LABELS = ["GT", "NN"]

FN_COLOR = "#2F8045"
IS_GT_COLOR = "#3E986A"
MATCHED_GT_COLOR = "#7EB489"

FP_COLOR = "#F39F1D"
IS_NN_COLOR = "#F6B619"
MATCHED_NN_COLOR = "#FACD45"

OTHER_ISLETS_COLOR = "#D0D0D0"

transparent_bg = {
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
}

In [ ]:
if COMPARISON_MODE:
    text = "# Results of model comparison (min. islet size: {}): \n".format(MIN_ISLET_SIZE)
    text += "## a. Model \"{}\"\n".format(MODEL_NAME[0])
    text += "## b. Model \"{}\"\n".format(MODEL_NAME[1])
else:
    text = "# Summary of results for model \"{}\" (min. islet size: {})".format(MODEL_NAME[0], MIN_ISLET_SIZE)
    
    
display(Markdown(text))

In [ ]:
@dataclass
class DataframesWithResults:
    image_df: pd.DataFrame
    matched_islets_df: pd.DataFrame
    is_islets_df: pd.DataFrame
    fn_islets_df: pd.DataFrame
    fp_islets_df: pd.DataFrame

def load_csvs(csv_root: str) -> DataframesWithResults:
    image_df = pd.read_csv(os.path.join(csv_root, IMAGE_CSV_FILE_NAME))
    matched_islets_df = pd.read_csv(os.path.join(csv_root, MATCHED_ISLETS_CSV_FILE_NAME))
    is_islets_df = pd.read_csv(os.path.join(csv_root, IS_ISLETS_CSV_FILE_NAME))
    fn_islets_df = pd.read_csv(os.path.join(csv_root, FN_ISLETS_CSV_FILE_NAME))
    fp_islets_df = pd.read_csv(os.path.join(csv_root, FP_ISLETS_CSV_FILE_NAME))
    
    return DataframesWithResults(image_df, matched_islets_df, is_islets_df, fn_islets_df, fp_islets_df)

def get_relative_array(nn_values: np.array, gt_values: np.array) -> np.array:
    nn_values = nn_values[gt_values > 0]
    gt_values = gt_values[gt_values > 0]
    return nn_values/gt_values

In [ ]:
def show_box_plot(boxes_values1: List[List[float]], boxes_values2: List[List[float]], boxes_names: List[str], title: str, legend_position: str = "top"):
    layout = go.Layout(
        height=500,
        legend={
            "xanchor": "right",
            "x": 1,
            "yanchor": "bottom",
            "y": 1.01,
            "orientation": "h",
        },
        title={
            "text": title
        },
        boxmode = "group" if COMPARISON_MODE else "overlay",
        showlegend=COMPARISON_MODE
    )
    fig = go.Figure(layout=layout)
    
    max_median = 0
    max_value = 0
    for values in boxes_values1 + boxes_values2 if COMPARISON_MODE else boxes_values1:
        max_median = max(np.median(values), max_median)
        max_value = max(*values, max_value)
    
    if max_value > 5 * max_median:
        fig.update_layout(xaxis=dict(type="log"))
    
    x = list()
    y = list()
    for values, name in zip(boxes_values1, boxes_names):
        x.extend(values)
        y.extend([name] * len(values))

    fig.add_trace(go.Box(x=x, y=y, name=MODEL_NAME[0]))
        
    if COMPARISON_MODE:
        x = list()
        y = list()
        for values, name in zip(boxes_values2, boxes_names):
            x.extend(values)
            y.extend([name] * len(values))
    
        fig.add_trace(go.Box(x=x, y=y, name=MODEL_NAME[1]))
        
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', minor_griddash="dot")

    fig.update_layout(transparent_bg)
    fig.update_traces(orientation='h')
    
    fig.show()

In [ ]:
@dataclass
class DistributionGraphAttributes:
    yaxis: str
    offsetgroup: int
    legendgrouptitle_text: str
    
DISTRIBUTION_BAR_ATTRIBUTES = [
    DistributionGraphAttributes(
        yaxis="y1",
        offsetgroup="0",
        legendgrouptitle_text="GT islets - " + MODEL_NAME[0],
    ),
    DistributionGraphAttributes(
        yaxis="y2",
        offsetgroup="1",
        legendgrouptitle_text="GT islets - " + MODEL_NAME[1],
    ),
    DistributionGraphAttributes(
        yaxis="y3" if COMPARISON_MODE else "y2",
        offsetgroup="2" if COMPARISON_MODE else "1",
        legendgrouptitle_text="NN islets - " + MODEL_NAME[0],
    ),
    DistributionGraphAttributes(
        yaxis="y4",
        offsetgroup="3",
        legendgrouptitle_text="NN islets - " + MODEL_NAME[1],
    ),
]

In [ ]:
def get_updated_islet_size_list(size_list: list, size_um: float) -> List[int]:
    for i in range(len(MIN_ISLET_SIZES)):
        if i == len(MIN_ISLET_SIZES) - 1:
            size_list[i] += 1
            break
            
        if size_um < MIN_ISLET_SIZES[i+1]:
            size_list[i] += 1
            break
    return size_list

def get_islet_count_per_size_single(df: pd.DataFrame) -> List[int]:
    count_per_size = [0] * len(ISLET_SIZES_LABELS)
    
    for size in df.size_um.values:
        count_per_size = get_updated_islet_size_list(count_per_size, size)
        
    return count_per_size
        
def get_islet_count_per_size_both(df: pd.DataFrame) -> Tuple[List[int], List[int]]:
    count_per_size_nn = [0] * len(ISLET_SIZES_LABELS)
    count_per_size_gt = [0] * len(ISLET_SIZES_LABELS)
    
    for gt_size in df.drop_duplicates(subset=["image_name", "islet_id_gt"]).size_um_gt.values:
        count_per_size_gt = get_updated_islet_size_list(count_per_size_gt, gt_size)
        
    for nn_size in df.drop_duplicates(subset=["image_name", "islet_id_nn"]).size_um_nn.values:
        count_per_size_nn = get_updated_islet_size_list(count_per_size_nn, nn_size)
        
    return count_per_size_gt, count_per_size_nn

In [ ]:
def get_size_distribution_bar(name: str, y: List[int], color: str, legendgroup: int, show_gt_and_nn: bool = True) -> go.Bar:
    bar_attributes = DISTRIBUTION_BAR_ATTRIBUTES[legendgroup]
    num_columns = 5 if COMPARISON_MODE else 3
    offset = legendgroup if COMPARISON_MODE else max(0, legendgroup-1)
    return go.Bar(
        name=name, 
        x=ISLET_SIZES_LABELS, 
        y=y, 
        yaxis=bar_attributes.yaxis, 
        offsetgroup=bar_attributes.offsetgroup if show_gt_and_nn else 0, 
        offset=(offset-1) * 1/num_columns if show_gt_and_nn else -0.4, 
        width=1/num_columns if show_gt_and_nn else 0.8, 
        marker_color=color,
        legendgroup=legendgroup, 
        legendgrouptitle_text=bar_attributes.legendgrouptitle_text
    )

def show_size_distriburion_chart(
    title: str,
    fn_count_per_size1: List[int] = list(),
    fp_count_per_size1: List[int] = list(),
    matched_gt_count_per_size1: List[int] = list(),
    matched_nn_count_per_size1: List[int] = list(),
    is_gt_count_per_size1: List[int] = list(),
    is_nn_count_per_size1: List[int] = list(),
    fn_count_per_size2: List[int] = list(),
    fp_count_per_size2: List[int] = list(),
    matched_gt_count_per_size2: List[int] = list(),
    matched_nn_count_per_size2: List[int] = list(),
    is_gt_count_per_size2: List[int] = list(),
    is_nn_count_per_size2: List[int] = list(),
    show_gt_and_nn: bool = True,
):
    fig = go.Figure(
        layout=go.Layout(
            barmode="relative",
            yaxis2=go.layout.YAxis(
                visible=False,
                matches="y",
                overlaying="y",
                anchor="x",
            ),
            yaxis3=go.layout.YAxis(
                visible=False,
                matches="y",
                overlaying="y",
                anchor="x",
            ),
            yaxis4=go.layout.YAxis(
                visible=False,
                matches="y",
                overlaying="y",
                anchor="x",
            ),
            height=600,
            title={"text": title}
        )

    )
    if len(fn_count_per_size1) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='False negative islets', y=fn_count_per_size1, color=FN_COLOR, legendgroup=0, show_gt_and_nn=show_gt_and_nn)
        )
    if len(is_gt_count_per_size1) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Incorrectly separated islets', y=is_gt_count_per_size1, color=IS_GT_COLOR, legendgroup=0, show_gt_and_nn=show_gt_and_nn)
        )
    if len(matched_gt_count_per_size1) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Matched islets', y=matched_gt_count_per_size1, color=MATCHED_GT_COLOR, legendgroup=0, show_gt_and_nn=show_gt_and_nn)
        )
        
    if len(fn_count_per_size2) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='False negative islets', y=fn_count_per_size2, color=FN_COLOR, legendgroup=1, show_gt_and_nn=show_gt_and_nn)
        )
    if len(is_gt_count_per_size2) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Incorrectly separated islets', y=is_gt_count_per_size2, color=IS_GT_COLOR, legendgroup=1, show_gt_and_nn=show_gt_and_nn)
        )
    if len(matched_gt_count_per_size2) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Matched islets', y=matched_gt_count_per_size2, color=MATCHED_GT_COLOR, legendgroup=1, show_gt_and_nn=show_gt_and_nn)
        )
    
    if len(fp_count_per_size1) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='False positive islets', y=fp_count_per_size1, color=FP_COLOR, legendgroup=2, show_gt_and_nn=show_gt_and_nn)
        )
    if len(is_nn_count_per_size1) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Incorrectly separated islets', y=is_nn_count_per_size1, color=IS_NN_COLOR, legendgroup=2, show_gt_and_nn=show_gt_and_nn)
        )
    if len(matched_nn_count_per_size1) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Matched islets', y=matched_nn_count_per_size1, color=MATCHED_NN_COLOR, legendgroup=2, show_gt_and_nn=show_gt_and_nn)
        )
        
    if len(fp_count_per_size2) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='False positive islets', y=fp_count_per_size2, color=FP_COLOR, legendgroup=3, show_gt_and_nn=show_gt_and_nn)
        )
    if len(is_nn_count_per_size2) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Incorrectly separated islets', y=is_nn_count_per_size2, color=IS_NN_COLOR, legendgroup=3, show_gt_and_nn=show_gt_and_nn)
        )
    if len(matched_nn_count_per_size2) > 0:
        fig.add_trace(
            get_size_distribution_bar(name='Matched islets', y=matched_nn_count_per_size2, color=MATCHED_NN_COLOR, legendgroup=3, show_gt_and_nn=show_gt_and_nn)
        )
    
    fig.show()

In [ ]:
def get_gt_nn_value_comparison_bar(
    name: str, values: [int, int], color: str, legendgroup: int, showlegend: bool = True
) -> go.Bar:
    bar_attributes = DISTRIBUTION_BAR_ATTRIBUTES[legendgroup]
    return go.Bar(
        name=name,
        y=GT_NN_LABELS,
        x=values,
        marker_color=color,
        showlegend=showlegend,
        legendgroup=legendgroup, 
        legendgrouptitle_text=bar_attributes.legendgrouptitle_text, 
        orientation="h"
    )

def show_gt_nn_value_comparison_bar_chart(
    title: str,
    fn_islets_value: int = None,
    fp_islets_value: int = None,
    matched_gt_islets_value: int = None,
    matched_nn_islets_value: int = None,
    is_gt_islets_value: int = None,
    is_nn_islets_value: int = None,
    other_gt_islets_value: int = None,
    other_nn_islets_value: int = None,
    small: bool = False,
    model_index: int = 0,
):
    fig = go.Figure(
        layout=go.Layout(
            barmode="stack",
            height=400 if not small else 320,
            title={"text": title},
        )
    )
    
    if fn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False negative islets", values=[fn_islets_value, 0], color=FN_COLOR, legendgroup=0 + model_index)
        )
    if is_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", values=[is_gt_islets_value, 0], color=IS_GT_COLOR, legendgroup=0 + model_index)
        )
    if matched_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", values=[matched_gt_islets_value, 0], color=MATCHED_GT_COLOR, legendgroup=0 + model_index)
        )
    if other_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other GT islets", values=[other_gt_islets_value, 0], color=OTHER_ISLETS_COLOR, legendgroup=0 + model_index)
        )
    
    if fp_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False positive islets", values=[0, fp_islets_value], color=FP_COLOR, legendgroup=2 + model_index)
        )
    if is_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", values=[0, is_nn_islets_value], color=IS_NN_COLOR, legendgroup=2 + model_index)
        )
    if matched_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", values=[0, matched_nn_islets_value], color=MATCHED_NN_COLOR, legendgroup=2 + model_index)
        )
    if other_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other NN islets", values=[0, other_nn_islets_value], color=OTHER_ISLETS_COLOR, legendgroup=2 + model_index)
        )


    fig.show()
    
def add_gt_nn_value_comparison_bar(
    fig: go.Figure,
    row: int,
    fn_islets_value: int = None,
    fp_islets_value: int = None,
    matched_gt_islets_value: int = None,
    matched_nn_islets_value: int = None,
    is_gt_islets_value: int = None,
    is_nn_islets_value: int = None,
    other_gt_islets_value: int = None,
    other_nn_islets_value: int = None,
): 
    showlegend = row == 1
    
    if fn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False negative islets", value=fn_islets_value, color=FN_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    if is_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", value=is_gt_islets_value, color=IS_GT_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    if matched_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", value=matched_gt_islets_value, color=MATCHED_GT_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    if other_gt_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other GT islets", value=other_gt_islets_value, color=OTHER_ISLETS_COLOR, legendgroup=0, showlegend=showlegend),
            col=1, row=row
        )
    
    if fp_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="False positive islets", value=fp_islets_value, color=FP_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
    if is_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Incorrectly separated islets", value=is_nn_islets_value, color=IS_NN_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
    if matched_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Matched islets", value=matched_nn_islets_value, color=MATCHED_NN_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
    if other_nn_islets_value is not None:
        fig.add_trace(
            get_gt_nn_value_comparison_bar(name="Other NN islets", value=other_nn_islets_value, color=OTHER_ISLETS_COLOR, legendgroup=1, showlegend=showlegend),
            col=1, row=row
        )
        
    return fig

def init_gt_nn_value_comparison_bar_chart():
    fig = make_subplots(
        rows=4, cols=1,
        row_heights=[0.25] * 4,
        specs=[[{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]],
        subplot_titles=[
            "Total islet count", 
            "Total islet area [μm]",
            "Total islet volume - Ellipse [IE]",
            "Total islet volume - Ricordi, short [IE]"  
        ],
        vertical_spacing=0.1,
        figure=go.Figure(
            layout = go.Layout(
                height=1000,
                barmode="stack",
            )
        )
    )
    return fig

In [ ]:
def get_max_iou_per_gt_islet(dfs: DataframesWithResults, only_is: bool = False):
    max_iou_per_gt_islet = list()

    # matched
    if not only_is:
        max_iou_per_gt_islet.extend(dfs.matched_islets_df.iou)

    # incorrectly separated
    for image_name in dfs.is_islets_df.image_name.unique():
        sub_df = dfs.is_islets_df.loc[dfs.is_islets_df.image_name == image_name]

        for gt_id in sub_df.islet_id_gt.unique():
            max_iou = sub_df.loc[sub_df.islet_id_gt == gt_id].iou.max()
            max_iou_per_gt_islet.append(max_iou)

    # false negative
    if not only_is:
        max_iou_per_gt_islet.extend([0.] * len(dfs.fn_islets_df))

    max_iou_per_gt_islet.sort(reverse=True)
    
    return max_iou_per_gt_islet

def plot_gt_ratio_iou_curve(
    max_iou_per_gt_islet1: List[float], 
    max_iou_per_gt_islet2: List[float], 
    gt_islets_ratio1: List[float],
    gt_islets_ratio2: List[float],
    islet_type: str,
):
    fig = go.Figure(
        layout=go.Layout(
            legend={
                    "xanchor": "right",
                    "x": 1,
                    "yanchor": "bottom",
                    "y": 1.01,
                    "orientation": "h",
                },
                showlegend=COMPARISON_MODE,
        )
    )
    
    fig.add_trace(go.Scatter(
        x=max_iou_per_gt_islet1, 
        y=gt_islets_ratio1, 
        mode='lines',
        name="{} (AUC = {:.4f})".format(MODEL_NAME[0], auc(max_iou_per_gt_islet1, gt_islets_ratio1)),
    ))
    if COMPARISON_MODE:
        fig.add_trace(go.Scatter(
            x=max_iou_per_gt_islet2, 
            y=gt_islets_ratio2, 
            mode='lines',
            name="{} (AUC = {:.4f})".format(MODEL_NAME[1], auc(max_iou_per_gt_islet2, gt_islets_ratio2)),
        ))
    
    fig.update_layout(
        title="Ratio of {} GT islets with max IoU higher than threshold".format(islet_type),
        xaxis_title="IoU threshold",
        yaxis_title="Ratio of incorrectly separated GT islets",
    )
    fig.show()
    
def plot_max_iou_per_gt_distribution(max_iou_per_gt_islet1: List[float], max_iou_per_gt_islet2: List[float]):
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.2, 0.8],
        specs=[[{"type": "box"}], [{"type": "histogram"}]],
        vertical_spacing=0.05,
        figure=go.Figure(
            layout = go.Layout(
                height=700,
                legend={
                    "xanchor": "right",
                    "x": 1,
                    "yanchor": "bottom",
                    "y": 1.01,
                    "orientation": "h",
                },
                showlegend=COMPARISON_MODE,
                title="Distribution of max IoU per GT islet",
                barmode="overlay" if COMPARISON_MODE else "group",
                boxmode="group" if COMPARISON_MODE else "overlay",
            ),
        )
    )
    
    fig.add_trace(go.Histogram(x=max_iou_per_gt_islet1, name=MODEL_NAME[0], nbinsx=50, marker_color="#636EFA"), row=2, col=1)
    
    if COMPARISON_MODE:
        fig.add_trace(go.Histogram(x=max_iou_per_gt_islet2, name=MODEL_NAME[1], nbinsx=50, marker_color="#EF553B"), row=2, col=1)
        fig.update_traces(opacity=0.7)
        
    fig.add_trace(go.Box(x=max_iou_per_gt_islet1, name='', marker_color="#636EFA", orientation='h', showlegend=False), row=1, col=1)
    if COMPARISON_MODE:
        fig.add_trace(go.Box(x=max_iou_per_gt_islet2, name='', marker_color="#EF553B", orientation='h', showlegend=False), row=1, col=1)
    
    fig.update_layout(
        xaxis2_title="Max IoU",
        yaxis2_title="Number of GT islets"
    )
    
    fig.show()

def show_gt_max_iou_graphs(max_iou_per_gt_islet1: List[float], max_iou_per_gt_islet2: List[float], islet_type: str):
    num_gt_islets1 = len(max_iou_per_gt_islet1)
    num_gt_islets2 = len(max_iou_per_gt_islet2) if COMPARISON_MODE else None

    y_data1 = [float(sum((np.array(max_iou_per_gt_islet1) >= 1)))] + [(i)/num_gt_islets1 for i in range(num_gt_islets1)] + [1.0]
    max_iou_per_gt_islet_extended1 = [1.0] + max_iou_per_gt_islet1 + [0.0]
    
    y_data2 = [float(sum((np.array(max_iou_per_gt_islet2) >= 1)))] + [(i)/num_gt_islets2 for i in range(num_gt_islets2)] + [1.0]  if COMPARISON_MODE else None
    max_iou_per_gt_islet_extended2 = [1.0] + max_iou_per_gt_islet2 + [0.0] if COMPARISON_MODE else None
    
    plot_gt_ratio_iou_curve(max_iou_per_gt_islet_extended1, max_iou_per_gt_islet_extended2, y_data1, y_data2, islet_type)
    plot_max_iou_per_gt_distribution(max_iou_per_gt_islet1, max_iou_per_gt_islet2)
    

In [ ]:
model1_dfs = load_csvs(CSVS_ROOT[0])
if COMPARISON_MODE:
    model2_dfs =load_csvs(CSVS_ROOT[1])

## 1. Semantic metrics

In [ ]:
boxes_names = ["IoU", "Precision", "Recall"]

boxes_values1 = [
    model1_dfs.image_df.iou,
    model1_dfs.image_df.precision,
    model1_dfs.image_df.recall,
]

boxes_values2 = [
    model2_dfs.image_df.iou,
    model2_dfs.image_df.precision,
    model2_dfs.image_df.recall,
] if COMPARISON_MODE else None

print("{} IoU: {} ± {}".format(MODEL_NAME[0], model1_dfs.image_df.iou.mean(), model1_dfs.image_df.iou.std()))
if COMPARISON_MODE:
    print("{} IoU: {} ± {}".format(MODEL_NAME[1], model2_dfs.image_df.iou.mean(), model2_dfs.image_df.iou.std()))
show_box_plot(boxes_values1, boxes_values2, boxes_names, "Metrics", legend_position="bottom")

## 2. Relative metrics

In [ ]:
attributes = ["islet_count", "total_area_um2", "total_volume_ellipse_ie"]
boxes_names = [
    "Relative islet count", 
    "Relative islet area", 
    "Relative islet volume<br> (Ellipse)", 
]

boxes_values1 = list()
boxes_values2 = list()
for attribute in attributes:
    relative_values = get_relative_array(
        nn_values=model1_dfs.image_df["{}_nn".format(attribute)],
        gt_values=model1_dfs.image_df["{}_gt".format(attribute)],
    )
    boxes_values1.append(relative_values)
    
    if COMPARISON_MODE:
        relative_values = get_relative_array(
            nn_values=model2_dfs.image_df["{}_nn".format(attribute)],
            gt_values=model2_dfs.image_df["{}_gt".format(attribute)],
        )
        boxes_values2.append(relative_values)

show_box_plot(boxes_values1, boxes_values2, boxes_names, "Relative values (NN/GT)")

## 3. Islet size distribution and type counts

In [ ]:
fn_islet_count_per_size1 = get_islet_count_per_size_single(model1_dfs.fn_islets_df)
fp_islet_count_per_size1 = get_islet_count_per_size_single(model1_dfs.fp_islets_df)
matched_gt_islet_count_per_size1, matched_nn_islet_count_per_size1 = get_islet_count_per_size_both(model1_dfs.matched_islets_df)
is_gt_islet_count_per_size1, is_nn_islet_count_per_size1 = get_islet_count_per_size_both(model1_dfs.is_islets_df)

fn_islets_count1 = np.sum(fn_islet_count_per_size1)
fp_islets_count1 = np.sum(fp_islet_count_per_size1)
matched_gt_islets_count1 = np.sum(matched_gt_islet_count_per_size1)
matched_nn_islets_count1 = np.sum(matched_nn_islet_count_per_size1)
is_gt_islets_count1 = np.sum(is_gt_islet_count_per_size1)
is_nn_islets_count1 = np.sum(is_nn_islet_count_per_size1)

if COMPARISON_MODE: 
    fn_islet_count_per_size2 = get_islet_count_per_size_single(model2_dfs.fn_islets_df)
    fp_islet_count_per_size2 = get_islet_count_per_size_single(model2_dfs.fp_islets_df)
    matched_gt_islet_count_per_size2, matched_nn_islet_count_per_size2 = get_islet_count_per_size_both(model2_dfs.matched_islets_df)
    is_gt_islet_count_per_size2, is_nn_islet_count_per_size2 = get_islet_count_per_size_both(model2_dfs.is_islets_df)
    
    fn_islets_count2 = np.sum(fn_islet_count_per_size2)
    fp_islets_count2 = np.sum(fp_islet_count_per_size2)
    matched_gt_islets_count2 = np.sum(matched_gt_islet_count_per_size2)
    matched_nn_islets_count2 = np.sum(matched_nn_islet_count_per_size2)
    is_gt_islets_count2 = np.sum(is_gt_islet_count_per_size2)
    is_nn_islets_count2 = np.sum(is_nn_islet_count_per_size2)

In [ ]:
if COMPARISON_MODE: 
    show_size_distriburion_chart(
        "Islet sizes distribution",
        fn_islet_count_per_size1,
        fp_islet_count_per_size1,
        matched_gt_islet_count_per_size1,
        matched_nn_islet_count_per_size1,
        is_gt_islet_count_per_size1,
        is_nn_islet_count_per_size1,
        fn_islet_count_per_size2,
        fp_islet_count_per_size2,
        matched_gt_islet_count_per_size2,
        matched_nn_islet_count_per_size2,
        is_gt_islet_count_per_size2,
        is_nn_islet_count_per_size2,
    )
else:
    show_size_distriburion_chart(
        "Islet sizes distribution",
        fn_islet_count_per_size1,
        fp_islet_count_per_size1,
        matched_gt_islet_count_per_size1,
        matched_nn_islet_count_per_size1,
        is_gt_islet_count_per_size1,
        is_nn_islet_count_per_size1,
    )

In [ ]:
show_gt_nn_value_comparison_bar_chart(
    "Total islet count - " + MODEL_NAME[0],
    fn_islets_value=fn_islets_count1,
    fp_islets_value=fp_islets_count1,
    matched_gt_islets_value=matched_gt_islets_count1,
    matched_nn_islets_value=matched_nn_islets_count1,
    is_gt_islets_value=is_gt_islets_count1,
    is_nn_islets_value=is_nn_islets_count1,
    model_index=0,
)

if COMPARISON_MODE: 
    show_gt_nn_value_comparison_bar_chart(
        "Total islet count - " + MODEL_NAME[1],
        fn_islets_value=fn_islets_count2,
        fp_islets_value=fp_islets_count2,
        matched_gt_islets_value=matched_gt_islets_count2,
        matched_nn_islets_value=matched_nn_islets_count2,
        is_gt_islets_value=is_gt_islets_count2,
        is_nn_islets_value=is_nn_islets_count2,
        model_index=1,
    )

## 4. All GT islets max IoU

In [ ]:
max_iou_per_gt_islet1 = get_max_iou_per_gt_islet(model1_dfs)
max_iou_per_gt_islet2 = get_max_iou_per_gt_islet(model2_dfs) if COMPARISON_MODE else None

show_gt_max_iou_graphs(max_iou_per_gt_islet1, max_iou_per_gt_islet2, islet_type="all")

## 5. Incorrectly separated GT islets count and max IoU

In [ ]:
is_gt_islets_count1 = np.sum(model1_dfs.image_df.incorrectly_separated_islets_count_gt)
is_nn_islets_count1 = np.sum(model1_dfs.image_df.incorrectly_separated_islets_count_nn)

fig = go.Figure(
    layout=go.Layout(
        title="Number of islets classified as incorrectly separated (GT vs NN)",
        xaxis_title="Number of islets",
        legend={
            "xanchor": "right",
            "x": 1,
            "yanchor": "bottom",
            "y": 1.01,
            "orientation": "h",
        },
        showlegend=COMPARISON_MODE,
        height=350,
    )
)
fig.add_trace(go.Bar(
    name=MODEL_NAME[0],
    y=["GT", "NN"],
    x=[is_gt_islets_count1, is_nn_islets_count1],
    orientation="h"
))

if COMPARISON_MODE:
    is_gt_islets_count2 = np.sum(model2_dfs.image_df.incorrectly_separated_islets_count_gt)
    is_nn_islets_count2 = np.sum(model2_dfs.image_df.incorrectly_separated_islets_count_nn)
    
    fig.add_trace(go.Bar(
        name=MODEL_NAME[1],
        y=["GT", "NN"],
        x=[is_gt_islets_count2, is_nn_islets_count2],
        orientation="h"
    ))
    
fig.update_traces(opacity=0.8)
fig.show()

In [ ]:
max_iou_per_gt_islet1 = get_max_iou_per_gt_islet(model1_dfs, only_is=True)
max_iou_per_gt_islet2 = get_max_iou_per_gt_islet(model2_dfs, only_is=True) if COMPARISON_MODE else None

show_gt_max_iou_graphs(max_iou_per_gt_islet1, max_iou_per_gt_islet2, islet_type="incorrectly separated")